#uncomment to install llama-index, make sure it is above 0.10.x
## TODO :
NVIDIARerank.get_available_models is currently hardcoded as dictionaly items

In [ ]:
from llama_index.postprocessor.nvidia_rerank import NVIDIARerank

NVIDIARerank.get_available_models()

In [ ]:
from llama_index.postprocessor.nvidia_rerank import NVIDIARerank

ranker = NVIDIARerank(top_n=5)

## establish the reranker class with the local nim, supplying a base_url

In [1]:
from llama_index.postprocessor.nvidia_rerank import NVIDIARerank
rerank=NVIDIARerank().mode(mode="nim", base_url="http://127.0.0.1:1976/v1")

## we can set top_n like this

In [2]:
## setting top_n since our examples dont have more than 5 
rerank.top_n=2
print(rerank.top_n)
rerank.max_batch_size = 32


2


In [3]:
from llama_index.core import Document
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter,SimpleFileNodeParser
# load documents
documents = SimpleDirectoryReader("/workspace/APICatalog_reranker/data").load_data()

# parse nodes
parser = SentenceSplitter(separator='\n', chunk_size=200, chunk_overlap=0)
nodes = parser.get_nodes_from_documents(documents)



In [ ]:
nodes[0].get_content , nodes[1].get_content

## load some toy data

In [ ]:
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.data_structs import Node
from llama_index.core.schema import NodeWithScore
query = "which way should i go?"
passages = [
    "two roads diverged in a yellow wood, and sorry i could not travel both and be one traveler, long i stood and looked down one as far as i could to where it bent in the undergrowth;",
    "then took the other, as just as fair, and having perhaps the better claim because it was grassy and wanted wear, though as for that the passing there had worn them really about the same,",
    "and both that morning equally lay in leaves no step had trodden black. oh, i marked the first for another day! yet knowing how way leads on to way i doubted if i should ever come back.",
    "i shall be telling this with a sigh somewhere ages and ages hense: two roads diverged in a wood, and i, i took the one less traveled by, and that has made all the difference."
]

nodes = [
    NodeWithScore(node=Node(text=passages[0]), score=0.7),
    NodeWithScore(node=Node(text=passages[1]), score=0.8)
]


## validate with NVreranker works

In [6]:
from llama_index.postprocessor.nvidia_rerank import NVIDIARerank
query="tell me about Sweden's geography?"
rerank.postprocess_nodes(nodes, query_str=query )


[NodeWithScore(node=TextNode(id_='110bf467-4e16-411b-b4a9-5b1192cd20f5', embedding=None, metadata={'file_path': '/workspace/APICatalog_reranker/data/Sweden.txt', 'file_name': 'Sweden.txt', 'file_type': 'text/plain', 'file_size': 123365, 'creation_date': '2024-04-24', 'last_modified_date': '2024-04-24'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='26e9f026-0115-43f4-b6be-f84117b91af7', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/workspace/APICatalog_reranker/data/Sweden.txt', 'file_name': 'Sweden.txt', 'file_type': 'text/plain', 'file_size': 123365, 'creation_date': '2024-04-24', 'last_modified_date': '2024-04-24'}, hash='f2937488cfafa01fbfa630608ec381098029b73d3e708bc59804eb5ab978c8d0

## switch to APICatalog , for that we need to set the NVIDIA_API_KEY

In [7]:
import getpass
import os

## API Key can be found by going to NVIDIA NGC -> AI Foundation Models -> (some model) -> Get API Code or similar.
## 10K free queries to any endpoint (which is a lot actually).

# del os.environ['NVIDIA_API_KEY']  ## delete key and reset
if os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    print("Valid NVIDIA_API_KEY already in environment. Delete to reset")
    nvapi_key=os.environ["NVIDIA_API_KEY"] 

else:
    nvapi_key = getpass.getpass("NVAPI Key (starts with nvapi-): ")
    assert nvapi_key.startswith("nvapi-"), f"{nvapi_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvapi_key


NVAPI Key (starts with nvapi-):  ······································································


In [8]:
from llama_index.postprocessor.nvidia_rerank import NVIDIARerank

my_key=os.environ["NVIDIA_API_KEY"]
rerank=NVIDIARerank().mode(mode="nvidia",api_key=my_key)

## query the reranker via API Catalog's reranker

In [9]:

rerank.postprocess_nodes(nodes, query_str=query )

[NodeWithScore(node=TextNode(id_='110bf467-4e16-411b-b4a9-5b1192cd20f5', embedding=None, metadata={'file_path': '/workspace/APICatalog_reranker/data/Sweden.txt', 'file_name': 'Sweden.txt', 'file_type': 'text/plain', 'file_size': 123365, 'creation_date': '2024-04-24', 'last_modified_date': '2024-04-24'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='26e9f026-0115-43f4-b6be-f84117b91af7', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/workspace/APICatalog_reranker/data/Sweden.txt', 'file_name': 'Sweden.txt', 'file_type': 'text/plain', 'file_size': 123365, 'creation_date': '2024-04-24', 'last_modified_date': '2024-04-24'}, hash='f2937488cfafa01fbfa630608ec381098029b73d3e708bc59804eb5ab978c8d0